In [1]:
import pandas as pd
import numpy as np

import re
import os
import glob
import gc

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

/home/jan/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# get data
train_data_files = glob.glob(os.path.join("../data/train_thetas/", "*.csv"))
train_data_files.sort(key=lambda f: int(re.sub('\D', '', f)))
train_data = pd.concat((pd.read_csv(f, header=None) for f in train_data_files), ignore_index=True) # shape (1600 * 15000, 11)

# print train_data.shape
print(train_data.shape)

target_train_files = glob.glob(os.path.join("../data/training_ground_truth_files", "*_ground"))
target_train_files.sort(key=lambda f: int(re.sub('\D', '', f)))
target_train = pd.concat((pd.read_csv(f, header=None, sep=' ', dtype="int8").stack() for f in target_train_files), ignore_index=True) # shape (1600 * 15000)

# print target_train.shape
print(target_train.shape)

(24000000, 11)
(24000000,)


In [3]:
# split train_data into train and validation

X_train, X_val, y_train, y_val = train_test_split(train_data, target_train, test_size=0.2, random_state=42)

# print X_train data type
print(type(X_train))

<class 'pandas.core.frame.DataFrame'>


In [6]:
torch_x_train = torch.Tensor(X_train)
print(type(torch_x_train))

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 1474560000000000 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
# predict using random forest

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=-1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)

print(f"random forest accuracy: {accuracy_score(y_val, y_pred)}")

# random forest accuracy: 0.8926345833333333

In [ ]:
def aIOU(predictions, gt):
    classes = np.unique(np.concatenate((np.unique(predictions), np.unique(gt))))
    IoUs = np.zeros(len(classes) - 1)
    i = 0
    for cls in classes:
        if cls != 0:
            preds_tmp = predictions == cls
            gt_tmp = gt == cls
            IoUs[i] = np.sum(np.logical_and(preds_tmp, gt_tmp)) / np.sum(np.logical_or(preds_tmp, gt_tmp))
            i += 1
    return np.mean(IoUs)

In [ ]:
print(f"random forest aIOU: {aIOU(y_val, y_pred)}")

# random forest aIOU: 0.005818729442472708